# Lecture 7 – Data Wrangling and EDA


In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 7)
pd.set_option('display.max_columns', 8)
pd.set_option('precision', 2)
# This option stops scientific notation for pandas
pd.set_option('display.float_format', '{:.2f}'.format)


OptionError: 'Pattern matched multiple keys'

---
<br/><br/>

## Mauna Loa CO2 data

CO2 concentrations have been monitored at Mauna Loa Observatory since 1958 ([website link](https://gml.noaa.gov/ccgg/trends/data.html)).




Before we even begin to load the data it often helps to understand a little about the high-level structure:

1. How big is the data file?
1. How is the data file formatted?
1. How do we read the data into pandas?

### 1. How big is the data?

I often like to start my analysis by getting a rough estimate of the size of the data.  This will help inform the tools I use and how I view the data.  If it is relatively small I might use a text editor or a spreadsheet to look at the data.  If it is larger, I might jump to more programmatic exploration or even used distributed computing tools.

However here we will use python tools to probe the file.

In [ ]:
co2_file = "data/co2_mm_mlo.txt"

In [ ]:
print(co2_file, "is", os.path.getsize(co2_file) / 1e6, "MB")

with open(co2_file, "r") as f:
    print(co2_file, "is", sum(1 for l in f), "lines.")

data/co2_mm_mlo.txt is 0.051131 MB
data/co2_mm_mlo.txt is 810 lines.


### How do we read the file into Pandas?
While we could use Python again to check out the contents, let's instead check out this file with JupyterLab (note it's a `.txt` file. Do we trust this file extension?).


Looking at the first few lines of the data, we spot some relevant characteristics:

- The values are separated by white space, possibly tabs.
- The data line up down the rows. For example, the month appears in 7th to 8th position of each line.
- The 71st and 72nd lines in the file contain column headings split over two lines.

We can use `read_csv` to read the data into a Pandas data frame, and we provide several arguments to specify that the separators are white space, there is no header (**we will set our own column names**), and to skip the first 72 rows of the file.

In [ ]:
co2 = pd.read_csv(
    co2_file,
    header = None,
    skiprows = 72, 
    sep = '\s+', # regex for continuous whitespace (next lecture)
)
co2.head()

,0,1,2,3,4,5,6
0,1958,3,1958.21,315.71,315.71,314.62,-1
1,1958,4,1958.29,317.45,317.45,315.29,-1
2,1958,5,1958.38,317.50,317.50,314.71,-1
3,1958,6,1958.46,-99.99,317.10,314.85,-1
4,1958,7,1958.54,315.86,315.86,314.98,-1


Congratulations! You've wrangled your first set of real world data!

<br/>

...But our columns aren't named.
**We need to do more EDA.**

### JSON
We have discussed how to deal with csv, tsv and txt file, now let us take a look at an example with JSON.
The City of New York Open Data [website](https://data.cityofnewyork.us/Environment/2018-Central-Park-Squirrel-Census-Hectare-Data/ej9h-v6g2) about [squirrel census](https://www.nytimes.com/interactive/2020/01/08/nyregion/central-park-squirrel-census.html).

---
### 1. How big is the data file?

In [ ]:
squirrel_file = 'data/squirrel.json'
print(squirrel_file, "is", os.path.getsize(squirrel_file) / 1e6, "MB")

with open(squirrel_file, "r") as f:
    print(squirrel_file, "is", sum(1 for l in f), "lines.")

data/squirrel.json is 0.195071 MB
data/squirrel.json is 1625 lines.


<br/><br/>
### 2. How is the data file formatted?

In [ ]:
# raw text
print(squirrel_file, "======================")
with open(squirrel_file, "r") as f:
    for i in range(20):
        print(f"{i:03} | {repr(f.readline())}") # zero pad line numbers

data/squirrel.json ======================
000 | '{\n'
001 | '  "meta" : {\n'
002 | '    "view" : {\n'
003 | '      "id" : "ej9h-v6g2",\n'
004 | '      "name" : "2018 Central Park Squirrel Census - Hectare Data",\n'
005 | '      "assetType" : "dataset",\n'
006 | '      "attribution" : "The Squirrel Census",\n'
007 | '      "attributionLink" : "https://www.thesquirrelcensus.com/",\n'
008 | '      "averageRating" : 0,\n'
009 | '      "category" : "Environment",\n'
010 | '      "createdAt" : 1571157309,\n'
011 | '      "description" : "The Squirrel Census (https://www.thesquirrelcensus.com/) is a multimedia science, design, and storytelling project focusing on the Eastern gray (Sciurus carolinensis). They count squirrels and present their findings to the public. This table contains environmental data related to each of the 350 “countable” hectares of Central Park. Examples include weather, litter, animals sighted, and human density.",\n'
012 | '      "displayType" : "table",\n'
013 | '    

This appears to be a fairly standard JSON file.  We notice that the file appears to contain a description of itself in a field called "meta" (which is presumably short for **metadata**).

<br/><br/>
### 3. How do we read the file into Pandas?

We'll come back to this question. Let's first understand more about the particular structure of this JSON file so that we can decide what (if anything) to load into Pandas. 



---
<br/><br/>

# Digging into JSON

Python has relatively good support for JSON data since it closely matches the internal python object model.  In the following cell we import the entire JSON datafile into a python dictionary.

In [ ]:
import json

with open(squirrel_file, "rb") as f:
    squirrel_json = json.load(f)

The `squirrel_json` variable is now a dictionary encoding the data in the file:

In [ ]:
type(squirrel_json)

dict

### Examine what keys are in the top level json object

We can list the keys to determine what data is stored in the object.

In [ ]:
squirrel_json.keys()

dict_keys(['meta', 'data'])

**Observation**: The JSON dictionary contains a `meta` key which likely refers to meta data (data about the data).  Meta data often maintained with the data and can be a good source of additional information.

<br/>

We can investigate the meta data further by examining the keys associated with the metadata.

In [ ]:
squirrel_json['meta'].keys()

dict_keys(['view'])

The `meta` key contains another dictionary called `view`.  This likely refers to meta-data about a particular "view" of some underlying database.  We will learn more about views when we study SQL later in the class.    

In [ ]:
squirrel_json['meta']['view'].keys()

dict_keys(['id', 'name', 'assetType', 'attribution', 'attributionLink', 'averageRating', 'category', 'createdAt', 'description', 'displayType', 'downloadCount', 'hideFromCatalog', 'hideFromDataJson', 'indexUpdatedAt', 'newBackend', 'numberOfComments', 'oid', 'provenance', 'publicationAppendEnabled', 'publicationDate', 'publicationGroup', 'publicationStage', 'rowsUpdatedAt', 'rowsUpdatedBy', 'tableId', 'totalTimesRated', 'viewCount', 'viewLastModified', 'viewType', 'approvals', 'clientContext', 'columns', 'grants', 'metadata', 'owner', 'query', 'rights', 'tableAuthor', 'tags', 'flags'])

In [ ]:
print(len(squirrel_json['meta']['view'].keys()))

40


Notice that this a nested/recursive data structure.  As we dig deeper we reveal more and more keys and the corresponding data:

```
meta
|-> data
    | ... (haven't explored yet)
|-> view
    | -> id
    | -> name
    | -> attribution 
    ...
    | -> description
    ...
    | -> columns
    ...
```

There is a key called description in the view sub dictionary.  This likely contains a description of the data:

In [ ]:
print(squirrel_json['meta']['view']['description'])

The Squirrel Census (https://www.thesquirrelcensus.com/) is a multimedia science, design, and storytelling project focusing on the Eastern gray (Sciurus carolinensis). They count squirrels and present their findings to the public. This table contains environmental data related to each of the 350 “countable” hectares of Central Park. Examples include weather, litter, animals sighted, and human density.


<br/>

### Columns Metadata

Another potentially useful key in the metadata dictionary is the `columns`.  This returns a list:

In [ ]:
type(squirrel_json['meta']['view']['columns'])

list

We can browse summary data in the list using python:

**Observations**

1. The above meta data tells us a lot about the columns in the data including column names, potential data anomalies, and a basic statistic. 
1. JSON makes it easier (than CSV) to create **self-documented data.** 
1. Self documenting data can be helpful since it maintains its own description and these descriptions are more likely to be updated as data changes. 


### Examining the Data Field for Records

We can look at a few entries in the `data` field. This is what we'll load into Pandas.


In [ ]:
for i in range(3):
    print(f"{i:03} | {squirrel_json['data'][i]}")

000 | ['row-xvbm_ap36.wksi', '00000000-0000-0000-4A7D-FFDE6DF1A798', 0, 1571157630, None, 1571157630, None, '{ }', '01A', 'AM', '10072018', '110', '70º F, Foggy', 'Some', None, 'Humans, Pigeons', 'Busy', None, '1', '4', '22']
001 | ['row-uuju.7aay~5rhx', '00000000-0000-0000-7AE3-75B7008F060F', 0, 1571157630, None, 1571157630, None, '{ }', '01A', 'PM', '10142018', '177', '54º F, overcast', 'Abundant', None, 'Humans, Pigeons', 'Busy', None, '1', '7', '26']
002 | ['row-xtx4_48mj.a4yd', '00000000-0000-0000-4098-680F6C4866EC', 0, 1571157630, None, 1571157630, None, '{ }', '01B', 'AM', '10122018', '11', '60º F, sunny', 'Some', None, 'Humans, Dogs, Pigeons, Horses', 'Busy', None, '1', '17', '23']


---
<br/>

### 3. How do we read the file into Pandas? Building a DataFrame from JSON

Let's understand more about the particular structure of this JSON file so that we can decide what (if anything) 

In the following block of code we:

Translate the JSON records into a dataframe:

    * fields: `squirrel_json['meta']['view']['columns']`
    * records: `squirrel_json['data']`
    

Examine the table.

In [ ]:
# Load the data from JSON and assign column titles
squirrel = pd.DataFrame(
    squirrel_json['data'],
    columns=[c['name'] for c in squirrel_json['meta']['view']['columns']])

squirrel.tail()

,sid,id,position,created_at,...,Hectare Conditions Notes,Number of sighters,Number of Squirrels,Total Time of Sighting
695,row-e8hy_kmwx-4x2k,00000000-0000-0000-0628-259F1071BAB6,0,1571157630,...,None,3,12,20
696,row-du6k-kkp8.g7kn,00000000-0000-0000-BF38-4CCF9397B6A1,0,1571157630,...,None,3,3,26
697,row-bei3-r3r3~4wnr,00000000-0000-0000-E657-914DF5544732,0,1571157630,...,None,3,7,29
698,row-hw87~p8jp-ypem,00000000-0000-0000-0F8F-6E33DAF78AF4,0,1571157630,...,None,3,8,30
699,row-ceie~ihp3.b6ta,00000000-0000-0000-5B6C-6456C090D8D1,0,1571157630,...,None,3,6,30


<br/><br/>
---
# Real World Example: Wrangling CO2 Measurements

Let's continue looking at the CO2 concentrations at Mauna Loa Observatory:

In [ ]:
co2.head()

NameError: name 'co2' is not defined

We need column names!

---
## Exploring Variable Feature Types
Let's go back to the raw data file to understand each feature.

The NOAA [webpage](https://gml.noaa.gov/ccgg/trends/) might have some useful tidbits (in this case it doesn't).

<br/><br/>
We'll rerun `pd.read_csv`, but this time with some **custom column names.**

In [ ]:
co2 = pd.read_csv(
    co2_file, header = None, skiprows = 72, 
    sep = '\s+', # regex for continuous whitespace (next lecture)
    names = ['Yr', 'Mo', 'DecDate', 'Avg', 'Int', 'Trend', 'Days']
)
co2.head()

<br/><br/>

---

## Let's start exploring!!

Scientific studies tend to have very clean data, right? Let's jump right in and make a time series plot of CO2 monthly averages.

In [ ]:
sns.lineplot(x='DecDate', y='Avg', data=co2);

The code above uses the `seaborn` plotting library (abbreviated `sns`).
We won't cover this library in detail until next week, so focus
on the plots themselves, not the code used to create them.


Yikes! Plotting the data uncovered a problem. It looks like we have some **missing values**. What happened here? 

In [ ]:
co2.head()

In [ ]:
co2.tail()

Some data have unusual values like -1 and -99.99.

Let's check the description at the top of the file again.
1. -1 signifies a missing value for the number of days `Days` the equipment was in operation that month.
1. -99.99 denotes a missing monthly average `Avg`

How can we fix this? First, let's explore other aspects of our data. Understanding our data will help us decide what to do with the missing values.

<br/>

---

## Quality Checks: Reasoning about the data
First, we consider the shape of the data. How many rows should we have?
* If chronological order, we should have one record per month.
* Data from March 1958 to August 2019.
* We should have $ 12 \times (2019-1957) - 2 - 4 = 738 $ records.

In [ ]:
co2.shape

Nice!! The number of rows (i.e. records) match our expectations.

<br/><br/>

---

Let's now check the quality of each feature.

## Understanding Missing Value 1: `Days`
`Days` is a time field, so let's analyze other time fields to see if there is an explanation for missing values of days of operation.

Let's start with **months** `Mo`.

Are we missing any records? The number of months should have 62 or 61 instances (March 1957-August 2019).

In [ ]:
co2["Mo"].value_counts().sort_index()

As expected Jan, Feb, Sep, Oct, Nov, and Dec have 61 occurrences and the rest 62.

<br/><br/>

Next let's explore **days** `Days` itself, which is the number of days that the measurement equipment worked.

In [ ]:
sns.displot(co2['Days']);
plt.title("Distribution of days feature")
plt.show() # suppresses unneeded plotting output

In terms of data quality, a handful of months have averages based on measurements taken on fewer than half the days. In addition, there are nearly 200 missing values--**that's about 27% of the data**!

<br/><br/>

Finally, let's check the last time feature, **year** `Yr`.

Let's check to see if there is any connection between missingness and the year of the recording.

In [ ]:
sns.scatterplot(x="Yr", y="Days", data=co2);
plt.title("Day field by Year"); # the ; suppresses output

**Observations**:

* All of the missing data are in the early years of operation.
* It appears there may have been problems with equipment in the mid to late 80s.

**Potential Next Steps**:
* Confirm these explanations through documentation about the historical readings.
* Maybe drop earliest recordings? However, we would want to delay such action until after we have examined the time trends and assess whether there are any potential problems.

---
<br/><br/>

## Understanding Missing Value 2: `Avg`
Next, let's return to the -99.99 values in `Avg` to analyze the overall quality of the CO2 measurements.

In [ ]:
# Histograms of average CO2 measurements
sns.displot(co2['Avg']);

The non-missing values are in the 300-400 range (a regular range of CO2 levels).

We also see that there are only a few missing `Avg` values (**<1% of values**). Let's examine all of them:

In [ ]:
co2[co2["Avg"] < 0]

There doesn't seem to be a pattern to these values, other than that most records also were missing `Days` data.

## Drop or Impute Missing `Avg` Data?

Remember we want to fix the following plot:

In [ ]:
sns.lineplot(x='DecDate', y='Avg', data=co2)
plt.title("CO2 Average By Month");

Since we are plotting `Avg` vs `DecDate`, we should just focus on dealing with missing values for `Avg`.

Let's consider a few options:
1. Drop those records
1. Replace -99.99 with NaN
1. Substitute it with a likely value for the average CO2?

What do you think are the pros and cons of each possible action?

---
<br/><br/>
Let's examine each of these three options.

In [ ]:
# 1. Drop missing values
co2_drop = co2[co2['Avg'] > 0]

# 2. Replace NaN with -99.99
co2_NA = co2.replace(-99.99, np.NaN)

We'll also use a third version of the data.
First, we note that the dataset already comes with a **substitute value** for the -99.99.

From the file description:

>  The `interpolated` column includes average values from the preceding column (`average`)
and **interpolated values** where data are missing.  Interpolated values are
computed in two steps...

The `Int` feature has values that exactly match those in `Avg`, except when `Avg` is -99.99, and then a **reasonable** estimate is used instead.
So, the third version of our data will use the `Int` feature instead of `Avg`.

In [ ]:
# 3. Use interpolated column which estimates missing Avg values
co2_impute = co2.copy()
co2_impute['Avg'] = co2['Int']

<br/>

---

What's a **reasonable** estimate?

To answer this question, let's zoom in on a short time period, say the measurements in 1958 (where we know we have two missing values).


In [ ]:
# results of plotting data in 1958

def line_and_points(data, ax, title):
    # assumes single year, hence Mo
    ax.plot('Mo', 'Avg', data=data)
    ax.scatter('Mo', 'Avg', data=data)
    ax.set_xlim(2, 13)
    ax.set_title(title)
    ax.set_xticks(np.arange(3, 13))

def data_year(data, year):
    return data[data["Yr"] == 1958]
    
# uses matplotlib subplots
# you may see more next week; focus on output for now
fig, axes = plt.subplots(ncols = 3, figsize=(12, 4), sharey=True)

year = 1958
line_and_points(data_year(co2_drop, year), axes[0], title="1. Drop Missing")
line_and_points(data_year(co2_NA, year), axes[1], title="2. Missing Set to NaN")
line_and_points(data_year(co2_impute, year), axes[2], title="3. Missing Interpolated")

fig.suptitle(f"Monthly Averages for {year}")
plt.tight_layout()

In the big picture since there are only 7 `Avg` values missing (**<1%** of 738 months), any of these approaches would work.

However there is some appeal to **option 3: Imputing**:
* Shows seasonal trends for CO2
* We are plotting all months in our data as a line plot

<br/>

---
Let's replot our original figure with option 3:

In [ ]:
sns.lineplot(x='DecDate', y='Avg', data=co2_impute)
plt.title("CO2 Average By Month, Imputed");

Looks pretty close to what we see on the NOAA [website](https://gml.noaa.gov/ccgg/trends/)!

## Presenting the data: A Discussion on Data Granularity

From the description:
* monthly measurements are averages of average day measurements.
* The NOAA GML website has datasets for daily/hourly measurements too.

The data you present depends on your research question.

**How do CO2 levels vary by season?**
* You might want to keep average monthly data.

**Are CO2 levels rising over the past 50+ years, consistent with global warming predictions?**
* You might be happier with a **coarser granularity** of average year data!

In [ ]:
co2_year = co2_impute.groupby('Yr').mean()
sns.lineplot(x='Yr', y='Avg', data=co2_year)
plt.title("CO2 Average By Year");

Indeed, we see a rise by nearly 100 ppm of CO2 since Mauna Loa began recording in 1958.